In [ ]:
!pip install transformers datasets

We will only take 1% of the data as this is not the actual model but a simple approach for developers to understand.

In [ ]:
from datasets import load_dataset
training_data = load_dataset('imdb', split='train[:1%]')
test_data = load_dataset('imdb', split='test[:1%]')

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
def tokenize_function(sample):
    return tokenizer(sample['text'], padding='max_length', truncation=True)

tokenized_training_data = training_data.map(tokenize_function, batched='True')
tokenized_test_data = test_data.map(tokenize_function, batched=True)

In [ ]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased', num_labels=2
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

In [ ]:
trainer.evaluate(tokenized_test_data)